In [2]:
import findspark
findspark.init('/home/nono/spark-2.4.0-bin-hadoop2.7')
from pyspark.sql import SparkSession


In [4]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [6]:
from pyspark.ml.regression import LinearRegression


In [9]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [10]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [11]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [12]:
lrModel = lr.fit(training)

In [13]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [15]:
lrModel.intercept


0.14228558260358093

In [16]:
training_summary = lrModel.summary

In [20]:
training_summary.rootMeanSquaredError

10.16309157133015

In [23]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [27]:
train_data, test_data = all_data.randomSplit([0.7,0.3])

In [25]:
split_object

[DataFrame[label: double, features: vector],
 DataFrame[label: double, features: vector]]

In [28]:
train_data.describe().show()


+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                343|
|   mean|0.04295359363611147|
| stddev|   10.4464924084643|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [29]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                158|
|   mean| 0.7213178310285069|
| stddev| 10.050002239808688|
|    min|-21.432387764165806|
|    max| 27.111027963108548|
+-------+-------------------+



In [30]:
correct_model = lr.fit(train_data)

In [31]:
test_results = correct_model.evaluate(test_data)

In [32]:
test_results.rootMeanSquaredError

9.925784934919516

In [33]:
unlabeled_data = test_data.select('features')

In [34]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [35]:
predictions = correct_model.transform(unlabeled_data)

In [36]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|-0.22961648965869996|
|(10,[0,1,2,3,4,5,...|   0.311379653227575|
|(10,[0,1,2,3,4,5,...|  0.5736894888258265|
|(10,[0,1,2,3,4,5,...|  0.6025135266130616|
|(10,[0,1,2,3,4,5,...| -2.0203263767514033|
|(10,[0,1,2,3,4,5,...| -1.6804452656683186|
|(10,[0,1,2,3,4,5,...|  -2.493584474635825|
|(10,[0,1,2,3,4,5,...|   1.168672137762753|
|(10,[0,1,2,3,4,5,...| 0.07776042380020161|
|(10,[0,1,2,3,4,5,...|  2.7707639378674283|
|(10,[0,1,2,3,4,5,...| -0.6392008843897571|
|(10,[0,1,2,3,4,5,...| -0.7173136314496262|
|(10,[0,1,2,3,4,5,...|  -1.522769987237989|
|(10,[0,1,2,3,4,5,...|  3.4697195664800193|
|(10,[0,1,2,3,4,5,...|  -3.622731923816982|
|(10,[0,1,2,3,4,5,...|  -2.644734477968797|
|(10,[0,1,2,3,4,5,...|  -3.759178684465828|
|(10,[0,1,2,3,4,5,...|  0.9232811105653481|
|(10,[0,1,2,3,4,5,...|  -2.429673516728467|
|(10,[0,1,2,3,4,5,...|-0.0645754